In [34]:
import numpy as np
import pandas as pd
import os
from PIL import Image
import glob
from sklearn.model_selection import train_test_split
import time
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping

In [19]:
image_list = []
genres = []
for filename in glob.glob('../images_original/*/*.png'): #assuming png
    im = np.array(Image.open(filename).convert('L'))/255 - 0.5
    genre = filename.split('/')[2]
    
    genres.append(genre)
    image_list.append(im)

In [20]:
unique_genres = list(np.unique(genres))
unique_genres

['blues',
 'classical',
 'country',
 'disco',
 'hiphop',
 'jazz',
 'metal',
 'pop',
 'reggae',
 'rock']

In [21]:
def map_genre_to_onehot(genre_str):
    '''
    Genre str: String containing genre name
    Returns: one hot encoding of genre string given list of classes
    '''
    ohe = np.zeros(shape=(10,))
    genre_idx = unique_genres.index(genre_str)
    ohe[genre_idx] = 1
    return ohe

In [22]:
def map_genre_to_number(genre_str):
    genre_idx = unique_genres.index(genre_str)
    return genre_idx

In [23]:
genres = pd.Series(genres)
genres_nums = genres.map(map_genre_to_number).values

In [24]:
y = to_categorical(genres_nums)
y.shape

(999, 10)

In [25]:
genres = pd.Series(genres)
genres_onehot = genres.map(map_genre_to_onehot)

In [31]:
X = np.asarray(image_list).reshape(999,288,432,1)

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [43]:
early_stopping = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)

In [44]:
#create model
model = tf.keras.Sequential()
#add model layers
model.add(Conv2D(10, kernel_size=2, activation='relu', input_shape=(288,432,1)))
model.add(MaxPooling2D(pool_size=4))
model.add(Conv2D(10, kernel_size=2, activation='relu'))
model.add(MaxPooling2D(pool_size=4))
model.add(Flatten())
model.add(Dropout(0.2))
model.add(Dense(10, activation='softmax'))

opt = tf.keras.optimizers.Adam(lr=1e-3)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=150, batch_size=8, verbose=1, callbacks=[early_stopping])

Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_34 (Conv2D)          (None, 287, 431, 10)      50        
                                                                 
 max_pooling2d_34 (MaxPoolin  (None, 71, 107, 10)      0         
 g2D)                                                            
                                                                 
 conv2d_35 (Conv2D)          (None, 70, 106, 10)       410       
                                                                 
 max_pooling2d_35 (MaxPoolin  (None, 17, 26, 10)       0         
 g2D)                                                            
                                                                 
 flatten_17 (Flatten)        (None, 4420)              0         
                                                                 
 dropout_17 (Dropout)        (None, 4420)            

In [47]:
preds = model.predict(X_test)